# <center>Traitement d'image du projet TouNum <br />Livrable 1</center>

![cesiLogo.jpg](cesiLogo.jpg)

|Auteur|Centre|Modification|
|---|---|---|
|A. RIGAUT|Arras|2020/12/15|
|B. THIBAULT|Arras|2020/12/15|
|R. VANCAMP|Arras|2020/12/15|
|T. POLY|Arras|2020/12/15|
|V. NAESSENS|Arras|2020/12/15|

</div>

## Context

L'entreprise TouNoum nous a chargé de traiter deux jeux d'images l'un bruité et le second flouté. Le but est de rendre les images plus traitables par un algorithme de machine learning.

## Jeux de données

Dans le dossier Dataset nous avons les deux jeux de données dans deux sous-dossiers, Noisy(bruité) et Blurry(flouté).
Tout d’abord, on charge les images grâce à la fonction imreadcollection() de la librairie skimage.io. Cela va nous permettre de charger toutes les images dans une collection, que l’on va séparer dans deux dossiers « noisy » et « blurry » en fonction de leur type.

In [65]:
import imageio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import os
from scipy import misc
from skimage.io import imread_collection, imsave
from skimage import *
from skimage.filters import unsharp_mask
import cv2
from skimage.restoration import estimate_sigma #Robust wavelet-based estimator of the (Gaussian) noise standard deviation


#creating a collection with the available images
noisy = imread_collection('Noisy/*.jpg')
blurry = imread_collection('Blurry/*.jpg')
noisyPost = []
blurryPost = []


## Test

Lors de nos phases de tests nous avons utilisé la librairie Scipy. Cette libarrie ne permet pas de traiter les images RGB(multichannel) nativement. Il y a donc deux démarches qui se proposaient à nous, passer l'image en niveau de gris ou séparer les canneaux RGB pour les traiter indépendamment.

Dans un premier temps, nous avons testé de passer d'une image RGB à une image en niveau de gris grâce à la fonction rgb2gray() puis d'appliquer la fonction singleChannelGaussFilter() qui permet d'appliquer un filtre gaussien sur une image monochannel mais les résultats ne nous convenaient pas car les images étant en niveaux de gris, il est préférable pour un algorithme de machine learning que celles-ci soient en couleurs.

Puis nous avons testé de traiter les canneux individuellement, avec la fonction splitColor nous avons séparé les trois canneaux RGB. Puis chacun de ces channels a été traité dans la fonction noisyTreatment(). Tout comme la phase de test précédente, les résultats ne nous convenaient pas, car certaines valeurs étaient aberrantes.

Nous avons donc choisi de changer de bibliothèque pour notre solution finale et d'utiliser scikit-image.

In [77]:
def rgb2gray(image):
    """Transform a polychrome image in a grey image
    Parameters
    ----------
    image : array-like, shape (width, height, channel)
        Data
        
    Returns
    -------
    Gray like image. One channel
    """
    r, g, b = image[:,:,0], image[:,:,1], image[:,:,2] #isolate each channel
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b #more detail about this scale: https://en.wikipedia.org/wiki/Grayscale
    return gray

def singleChannelGaussFilter(image, sig, vrange):
    """Perform action in a single channel of an image
    Parameters
    ----------
    image : array-like, shape (width, height, channel)
        Data
    sig : int
        Standard deviation
    vrange : array-like, shape (minus, plus)
        
    Returns
    -------
    Image processed
    """
    
    blurred = ndimage.gaussian_filter(image, sigma=sig, mode='reflect') #apply gaussian filter
    result = image + (image - blurred) * 1
    if vrange is not None:
        return np.clip(result, vrange[0], vrange[1], out=result) #Given an interval, values outside the interval are clipped to the interval edges. For example, if an interval of [0, 1] is specified, values smaller than 0 become 0, and values larger than 1 become 1.
    return result


def splitColor(image):
    """Create a tiple witch each channel
    Parameters
    ----------
    image : array-like, shape (width, height, channel)
        Data
        
    Returns
    -------
    Tuple (red channel, green channel, blue channel)
    """
    red = image[:, :, 0] #all x and y in channel 0 whose is red
    green = image[:, :, 1]
    blue = image[:, :, 2]
    return(red, green, blue)


def noisyTreatment(channel):
    """Create a tiple witch each channel
    Parameters
    ----------
    channel : array-like, shape (width, height, channel)
        Data
        
    Returns
    -------
    array-like, shape (width, height)
    """
    arr = []
    
    filter0 = np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])
    for layer in layers:
        gauss_denoised = ndimage.gaussian_filter(channel, sigma=2.5)
        channelFlou = scipy.ndimage.convolve(gauss_denoised, filter0)
        result = channel + (channel - channelFlou) * 1
        arr.append(result)

    return np.dstack((arr[0], arr[1], arr[2]))

## Solution

### explication

Dans cette partie nous avons utilisé la librairie Scikit-image.

Les deux fonctions effectuent un traitement similaire.

Pour chaque image contenue dans le dossier Noisy ou Blurry, nous appliquons la fonction unsharp_mask(). Puis une fois celle-ci traitée, nous l'enregistrons dans le dossier noisyEnd ou dans le dossier blurryEnd selon le type de l'image traitée.

### unsharp_mask

Dans un premier temps la fonction unsharp_mask() floute l'image de base selon le rayon donné en paramètre de la fonction. Le rayon correspond à la taille de la matrice appliquée pour le flou, dans notre cas la matrice est la suivante :

![title](matrice_gauss.png)



Dans un second temps la formule suivante est appliquée : 

enhanced image = original + amount * (original - blurred)

Amount correspond à l'amplification des details de l'image, plus celle-ci est élevée, plus les contrastes sont élevés.

In [77]:
def process_noisy(collection):  
    for file in range(len(collection)):
        result = unsharp_mask(collection[file], radius=1, amount=1)
        noisyPost.append(result)
        imsave('noisyEnd/' + collection.files[file][-7:-4] + '.jpg', img_as_ubyte(result))
    return 1
        
def process_blurry(collection):  
    for file in range(len(collection)):
        result = unsharp_mask(collection[file], radius=1, amount=1)
        blurryPost.append(result)
        imsave('blurryEnd/' + collection.files[file][-7:-4] + '.jpg', img_as_ubyte(result))
    return 1

In [78]:
process_noisy(noisy)
process_blurry(blurry)

1

## Indice de performance

### Images bruitées

Dans cette partie de notre algorithme le but est de prouver l'efficacité de notre traitement d'image.

Pour se faire nous avons utilisé la fonction estimate_sigma() de la biliothèque scikit-image. 

Cette fonction permet d'estimer l'écart type de la fonction de Gauss. Puis nous faisons une moyenne de tous les écarts types des images de base et une moyenne des écarts type des images traitées. Enfin, elle détermine le taux de variation entre les deux moyennes obtenues. La fonction nous renvoie un résultat avoisinant les 40%, ce qui signifie que la fonction de débruitage est performante.

In [81]:
# Get the Denoised images
noisyTreated = imread_collection('noisyEnd/*.jpg')

def estimate_noise(listNoise, listDeNoise):
    listSigmaNoise = []
    listSigmaDenoise = []
    # Get the noise sigma of each image then add it to the list
    for imageNoise in listNoise:
        sigmaNoise = estimate_sigma(imageNoise, multichannel=True, average_sigmas=True)
        listSigmaNoise.append(sigmaNoise)
    
    # Calculation of the mean sigma of the noised images
    totalSigmaNoise = sum(listSigmaNoise)
    meanSigmaNoise = totalSigmaNoise/len(listNoise)
    
    # Same action here for denoised images
    for imageDenoise in listDeNoise:
        sigmaDenoise = estimate_sigma(imageDenoise, multichannel=True, average_sigmas=True)
        listSigmaDenoise.append(sigmaDenoise)

    totalSigmaDenoise = sum(listSigmaDenoise)
    meanSigmaDenoise = totalSigmaDenoise/len(listDeNoise)

    return(meanSigmaNoise,meanSigmaDenoise)

# Calculation of the percentage difference between the sigma to obtain the performance index
meanSigmaNoise,meanSigmaDenoise = estimate_noise(noisy,noisyTreated)
performance_index_noisy = ((meanSigmaDenoise - meanSigmaNoise)/meanSigmaNoise)*100
print(round(performance_index_noisy,2))

39.91


### Images floutées

Pour connaitre la performance de notre processus, nous avons dû détecter le flou dans notre image originale. Pour cela, nous utilisons la variance de Laplace grâce à « cv2.Laplacian(collectionPre[file], cv2.CV_64F).var() ». Plus le résultat est proche de 0, plus l’image est floutée. Donc nous l’appliquons sur notre image de base puis sur notre image après traitement et nous ajoutons ces données dans un dataframe pour que cela soit plus lisible et exploitable par la suite. Comme vous pouvez le voir le résultat de la fonction appliquée aux images post-traitement est proche de zero, ce qui montre que le traitement est un succès.

In [70]:
def blurryDetection(collectionPre, collectionPost):  
    Df= pd.DataFrame(columns=['FileName','Blur rate pre-treatment','Blur rate post-treatment'])
    for file in range(len(collectionPre)):
        imagePre = cv2.Laplacian(collectionPre[file], cv2.CV_64F).var()
        imagePost = cv2.Laplacian(collectionPost[file], cv2.CV_64F).var()
        Df=Df.append({'FileName':collectionPre.files[file],'Blur rate pre-treatment':imagePre,'Blur rate post-treatment':imagePost},ignore_index=True)
    return Df

In [71]:
blurryDetection(blurry, blurryPost)

,FileName,Blur rate pre-treatment,Blur rate post-treatment
0,Blurry\blurry_001.jpg,417.029056,0.017327
1,Blurry\blurry_002.jpg,65.037823,0.002757
2,Blurry\blurry_003.jpg,607.788792,0.023722
3,Blurry\blurry_004.jpg,245.151501,0.010351
4,Blurry\blurry_005.jpg,227.685810,0.009605
...,...,...,...
145,Blurry\blurry_146.jpg,137.893416,0.005506
146,Blurry\blurry_147.jpg,262.208983,0.002033
147,Blurry\blurry_148.jpg,89.037813,0.007688
148,Blurry\blurry_149.jpg,112.238551,0.009539
